# Lab 12 Examples (Dijkstra's Single Source Shortest Path)

Click Shift+Enter in each code cell to run the code. Be sure to start with the `#include` directives to load the required libraries.

In [1]:
// For Lab 12, we are limited to using only the following #include directives:

#include <iostream>
#include <algorithm>
#include <cmath>
#include <functional> // std::greater
#include <queue>
#include <string>
#include <utility>
#include <vector>

## Overview

In today's lab, we will implement Dijkstra's single source shortest path algorithm. This algorithm finds the shortest paths from a source node to all other nodes in a weighted graph with non-negative edge weights. The graph can be directed or undirected, but it must not contain negative weight edges.

## Representing an Undirected Graph

Graphs are structures made up of nodes (vertices) connected by edges. They can be represented in code using various methods, such as adjacency matrices or adjacency lists. Below is a simple undirected graph. We can see this graph represented in code using both an adjacency matrix and an adjacency list.

<div style="text-align:center">
<img src="https://latessa.github.io/cpp-labs/images/Lab12/undirected.svg" alt="Undirected graph (A–D)" style="width:70%;height:auto;"/>
</div>


In [2]:
int n = 4;

std::vector<std::vector<int>> adjList = {
    {1, 2, 3}, // A: B C D
    {0, 2},    // B: A C
    {0, 1, 3}, // C: A B D
    {0, 2}     // D: A C
};

std::vector<std::vector<int>> adjMatrix = {
//   A  B  C  D
    {0, 1, 1, 1}, // A
    {1, 0, 1, 0}, // B
    {1, 1, 0, 1}, // C
    {1, 0, 1, 0}  // D
};

// Print adjacency list (with letter labels)
std::cout << "Adjacency List:\n";
for (int i = 0; i < n; ++i) {
    std::cout << char('A' + i) << ": ";
    for (int v : adjList[i]) 
        std::cout << char('A' + v) << ' ';
    std::cout << '\n';
}

// Print adjacency matrix
std::cout << "\nAdjacency Matrix:\n";
std::cout << "  ";
for (int i = 0; i < n; ++i) 
    std::cout << char('A' + i) << ' ';
std::cout << '\n';

for (int i = 0; i < n; ++i) {
    std::cout << char('A' + i) << ' ';
    for (int j = 0; j < n; ++j) std::cout << adjMatrix[i][j] << ' ';
        std::cout << '\n';
}


Adjacency List:
A: B C D 
B: A C 
C: A B D 
D: A C 

Adjacency Matrix:
  A B C D 
A 0 1 1 1 
B 1 0 1 0 
C 1 1 0 1 
D 1 0 1 0 


## Representing a Directed Graph

The graphs above are undirected. But directed graphs can be represented similarly.  In the matrix representation, a directed edge from node A to node B would be represented by a 1 in the row for A and the column for B, but a 0 in the row for B and the column for A.

<div style="text-align:center">
<img src="https://latessa.github.io/cpp-labs/images/Lab12/directed.svg" alt="Directed graph (A→B, A→C, B→C)" style="width:70%;height:auto;"/>
</div>

In [3]:
int n = 4;

// Adjacency list for a directed graph:
// A -> B, C; B -> C; C -> D; D -> (none)
std::vector<std::vector<int>> adjList = {
    {1, 2}, // A -> B, C
    {2},    // B -> C
    {3},    // C -> D
    {}      // D -> none
};

// Adjacency matrix for the same directed graph
//   A B C D
std::vector<std::vector<int>> adjMatrix = {
    {0,1,1,0}, // A
    {0,0,1,0}, // B
    {0,0,0,1}, // C
    {0,0,0,0}  // D
};

// Print adjacency list (with letter labels)
std::cout << "Directed Adjacency List:\n";
for (int i = 0; i < n; ++i) {
    std::cout << char('A' + i) << ": ";
    for (int v : adjList[i])
        std::cout << char('A' + v) << ' ';
    std::cout << '\n';
}

// Print adjacency matrix
std::cout << "\nDirected Adjacency Matrix:\n";
std::cout << "  ";
for (int i = 0; i < n; ++i) 
    std::cout << char('A' + i) << ' ';
std::cout << '\n';

for (int i = 0; i < n; ++i) {
    std::cout << char('A' + i) << ' ';
    for (int j = 0; j < n; ++j) std::cout << adjMatrix[i][j] << ' ';
    std::cout << '\n';
}

Directed Adjacency List:
A: B C 
B: C 
C: D 
D: 

Directed Adjacency Matrix:
  A B C D 
A 0 1 1 0 
B 0 0 1 0 
C 0 0 0 1 
D 0 0 0 0 


### Representing a Weighted Graph

In a weighted graph, each edge has an associated weight (or cost). 

In the adjacency matrix representation, instead of using 1s and 0s to indicate the presence or absence of edges, we use the actual weights. If there is no edge between two nodes, we can use a special value (like infinity) to indicate that.

In the adjacency list representation, each entry in the list can be a pair consisting of the neighboring node and the weight of the edge connecting them. The C++ standard library provides the `std::pair` container class, which is useful for key-value pairs or data with distinctly two parts such as a destination node and edge weight.

## Weighted Undirected Graph (with positive edge weights)

<div style="text-align:center">
<img src="https://latessa.github.io/cpp-labs/images/Lab12/undirected_weighted.svg" alt="Weighted undirected graph (A–D)" style="width:70%;height:auto;"/>
</div>

### Exploring the STL Pair Type

In [4]:
// Introducing the use of the PAIR type

// make_pair deduces the types automatically
auto p = std::make_pair(2, 5.54);
std::cout << "First: " << p.first << ", Second: " << p.second << '\n';

// Explicitly specifying the PAIR type
std::pair<double, int> p2;
p2.first = 12.45;
p2.second = 20;
std::cout << "First: " << p2.first << ", Second: " << p2.second << '\n';

std::vector<std::pair<int, double>> test;
test.push_back(std::make_pair(1, 2.5));

// Using initializer list to create a pair
test.push_back({3, 4.5});
for (const auto& pr : test) {
    std::cout << "First: " << pr.first << ", Second: " << pr.second << '\n';
}

First: 2, Second: 5.54
First: 12.45, Second: 20
First: 1, Second: 2.5
First: 3, Second: 4.5


In [5]:
int n = 4;
const int INF = 1e9; // A large value representing "infinity"

// Adjacency list
// pair<neighbor, weight>
std::vector<std::vector<std::pair<int,int>>> adjList = {
    {{1,4}, {2,1}, {3,7}}, // A - B(4), C(1), D(7)
    {{0,4}, {2,2}},        // B - A(4), C(2)
    {{0,1}, {1,2}, {3,3}}, // C - A(1), B(2), D(3)
    {{0,7}, {2,3}}         // D - A(7), C(3)
};

// Adjacency matrix
// INF = no edge
std::vector<std::vector<int>> adjMatrix = {
//      A    B    C    D
    {   0,   4,   1,   7}, // A
    {   4,   0,   2, INF}, // B
    {   1,   2,   0,   3}, // C
    {   7, INF,   3,   0}  // D
};

// Print adjacency list (with letter labels and weights)
std::cout << "Undirected Weighted Adjacency List:\n";
for (int i = 0; i < n; ++i) {
    std::cout << char('A' + i) << ": ";
    for (const auto &p : adjList[i]) {
        int v = p.first;
        int w = p.second;
        std::cout << char('A' + v) << "(" << w << ") ";
    }
    std::cout << '\n';
}

// Print adjacency matrix
std::cout << "\nUndirected Weighted Adjacency Matrix (0 = no edge):\n";
std::cout << "  ";
for (int i = 0; i < n; ++i) 
    std::cout << char('A' + i) << ' ';
std::cout << '\n';

for (int i = 0; i < n; ++i) {
    std::cout << char('A' + i) << ' ';
    for (int j = 0; j < n; ++j)
        if (adjMatrix[i][j] == INF)
            std::cout << "∞" << ' ';
        else
            std::cout << adjMatrix[i][j] << ' ';
    std::cout << '\n';
}

Undirected Weighted Adjacency List:
A: B(4) C(1) D(7) 
B: A(4) C(2) 
C: A(1) B(2) D(3) 
D: A(7) C(3) 

Undirected Weighted Adjacency Matrix (0 = no edge):
  A B C D 
A 0 4 1 7 
B 4 0 2 ∞ 
C 1 2 0 3 
D 7 ∞ 3 0 


### Choosing Between an Adjacency Matrix and an Adjacency List

**Adjacency Matrix**

* Best for dense graphs (many edges relative to nodes)
* Fast edge queries (checking if an edge exists between two nodes)
* Requires $O(V^2)$ space (could be wasteful for sparse graphs)

**Adjacency List**

* Best for sparse graphs (few edges relative to nodes)
* Requires $O(V + E)$ space (more space efficient for sparse graphs)
* Slower edge queries (checking if an edge exists between two nodes)

#### Which might work best for Dijkstra's algorithm?

Use an adjacency list.

* In the algorithm, we will be examining neighbors of a node.
* An adjacency list is a list of neighbors for each node.
* We can therefore iterate over a row of the adjacency list in $O(d)$ time, where $d$ is the degree of the node. This is because the row only contains the neighbors of the node.

If we used an adjacency matrix, we would need to iterate over all $V$ nodes in the matrix, which would take $O(V)$ time.

### Priority Queues (Heaps)

A **priority queue** is a data structure that stores elements arranged by priority. It allows for fast removal of the element with the highest (or lowest) priority. Unlike a normal FIFO `queue` which removes elements in the order they were inserted, a priority queue removes elements by order of their priority.

A **priority queue** is also called a *heap*. Specifically, there are max-heaps and min-heaps:

* **Max-heap**: top() returns the largest element (default behavior of `std::priority_queue`).
* **Min-heap**: top() returns the smallest element (use `std::greater` as the comparator).

Time complexity:

* **push / enqueue**: $O(\log n)$ (inserting and re-heapifying),
* **pop / dequeue**: $O(\log n)$ (removing top and re-heapifying),
* **top / peek**: $O(1)$.

The C++ Standard template library provides a `std::priority_queue`. Instead of implementing our own heap, which could be an entirely different lab, we will use the STL priority queue in our Dijkstra's algorithm implementation.

In [6]:
// std::priority_queue Example

// Default: max-heap (largest element on top)
std::priority_queue<int> maxHeap;
maxHeap.push(10);
maxHeap.push(5);
maxHeap.push(20);
std::cout << "Max top: " << maxHeap.top() << '\n'; // 20
maxHeap.pop(); // removes 20
std::cout << "Max top: " << maxHeap.top() << '\n'; // 10
maxHeap.pop(); // removes 10

// Min-heap using std::greater comparator
std::priority_queue<int, std::vector<int>, std::greater<int>> minHeap;
minHeap.push(10);
minHeap.push(5);
minHeap.push(20);
std::cout << "Min top: " << minHeap.top() << '\n'; // 5
minHeap.pop(); // removes 5
std::cout << "Min top: " << minHeap.top() << '\n'; // 10
minHeap.pop(); // removes 10

// Common pattern for Dijkstra: min-heap of pairs (distance, node)
// so the pair with the smallest distance is at the top.

std::priority_queue<std::pair<int,int>, std::vector<std::pair<int,int>>, std::greater<std::pair<int,int>>> pq;
pq.push({2, 1}); // (distance, node)
pq.push({0, 5});
pq.push({1, 4});
// pq.top() gives the smallest-distance pair (distance, node)
// to break ties, it compares the second element of the pair 
auto pr = pq.top(); 
std::cout << "Top pair: (" << pr.first << pr.second << ")\n";
pq.pop();


Max top: 20
Max top: 10
Min top: 5
Min top: 10
Top pair: (05)


## Dijkstra's Algorithm (Single-Source Shortest Path)

Dijkstra's algorithm finds the shortest paths from a single source node to all other nodes in a graph with non-negative edge weights. We will implement our algorithm using an adjacency list and a priority queue (min-heap) to efficiently select the next vertex with the smallest tentative distance.

### Dijkstra's Single Source Shortest Path is a Greedy Algorithm

Dijkstra's algorithm belongs to a class of algorithms known as **greedy algorithms**. Greedy algorithms build up a solution piece by piece, always choosing the next piece that offers the most immediate benefit. Greedy algorithms are typically applied to optimization problems. An optimization problem is one where we want to find the best solution among many possible solutions, such as finding the shortest path, the maximum value, or the least cost.

Not all optimization problems can be solved greedily. However, Dijkstra's algorithm works correctly for graphs with non-negative edge weights because it always expands the least costly node first, ensuring that once a node's shortest path is determined, it cannot be improved by any other path. Consequently, there is no need to backtrack or reconsider previously made decisions, which is a benefit of greedy algorithms.

### Time Complexity (using adjacency list + min-heap)

* Time: $O((V + E) \\log V)$ when using a binary heap (`priority_queue`).

We will now step through the algorithm using the following slides: [dijkstra_slides](https://latessa.github.io/cpp-labs/slides/Lab12/dijkstra_slides.pdf).
